In [ ]:
import casadi
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from matplotlib.animation import FuncAnimation

In [ ]:
# System parameters
mc = 1.0  # mass of cart
mp = 0.5  # mass of pole
l = 1.0  # length of pole (to center of mass)
g = 9.81  # gravity

# Define optimization variables
T = 3.0  # Time horizon
N = 100  # Number of control intervals
dt = T / N

# define dynamics
nq = 2  # [cart position, pole angle]
nv = 2  # [cart velocity, pole angular velocity]
cq = casadi.SX.sym("q", nq)
cv = casadi.SX.sym("v", nv)
cu = casadi.SX.sym("u", 1)  # force on cart

x, theta = cq[0], cq[1]
dx, dtheta = cv[0], cv[1]
F = cu[0]

sin_theta = casadi.sin(theta)
cos_theta = casadi.cos(theta)

# Mass matrix
M11 = mc + mp
M12 = mp * l * cos_theta
M21 = mp * l * cos_theta
M22 = mp * l**2

# Nonlinear terms
C1 = -mp * l * dtheta**2 * sin_theta
C2 = mp * g * l * sin_theta

# Calculate accelerations using the manipulator equation
det = M11 * M22 - M12 * M21
invM11 = M22 / det
invM12 = -M12 / det
invM21 = -M21 / det
invM22 = M11 / det

# Acceleration equations
ddx = invM11 * (F - C1) + invM12 * (-C2)
ddtheta = invM21 * (F - C1) + invM22 * (-C2)

aba_fn = casadi.Function("aba_fn", [cq, cv, cu], [casadi.vertcat(ddx, ddtheta)])


# Euler integrator
def euler_integrate(q, v, u):
    q_next = q + v * dt
    v_next = v + aba_fn(q, v, u) * dt
    return q_next, v_next


# Set up optimization
opti = casadi.Opti()

Q = opti.variable(nq, N + 1)
V = opti.variable(nv, N + 1)
U = opti.variable(1, N)

# Set initial state
opti.subject_to(Q[:, 0] == np.array([0, 0]))  # cart at origin, pole down
opti.subject_to(V[:, 0] == np.array([0, 0]))  # starting from rest

# Set final state
opti.subject_to(Q[:, N] == np.array([0, np.pi]))  # cart at origin, pole up
opti.subject_to(V[:, N] == np.array([0, 0]))  # ending at rest

# Set dynamics constraints
for k in range(N):
    q_next, v_next = euler_integrate(Q[:, k], V[:, k], U[:, k])
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

# Set control and state limits
force_limit = 10.0
pos_limit = 10.0
opti.subject_to(opti.bounded(-force_limit, U, force_limit))
opti.subject_to(opti.bounded(-pos_limit, Q[0, :], pos_limit))  # cart position limits

# Define objective
obj = 0
for k in range(N):
    obj += U[:, k].T @ U[:, k]  # control effort
    obj += (Q[0, k]) ** 2  # penalize cart deviation from center
    obj += 10 * (Q[1, k] - np.pi) ** 2  # penalize pole deviation from upright

opti.minimize(obj)

# Set solver options
p_opts = {"expand": True}
s_opts = {"max_iter": 1000}
opti.solver("ipopt", p_opts, s_opts)

# Solve
sol = opti.solve()

# Extract solution
Q_traj = sol.value(Q)
V_traj = sol.value(V)

# Animation
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(111)
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect("equal")
ax.grid(True)

# Create objects for animation
cart_width = 0.3
cart_height = 0.2
cart = plt.Rectangle((-cart_width / 2, -cart_height / 2), cart_width, cart_height, fill=True, color="blue")
(pole,) = ax.plot([], [], "r-", linewidth=3)
(mass,) = ax.plot([], [], "ro", markersize=10)


def update(frame):
    # Update cart position
    cart_x = Q_traj[0, frame]
    cart.set_x(cart_x - cart_width / 2)

    # Update pole position
    theta = Q_traj[1, frame]
    pole_x = [cart_x, cart_x + l * np.sin(theta)]
    pole_y = [0, -l * np.cos(theta)]
    pole.set_data(pole_x, pole_y)
    mass.set_data([pole_x[1]], [pole_y[1]])

    ax.add_patch(cart)
    return cart, pole, mass


anim = FuncAnimation(fig, update, frames=N + 1, interval=dt * 1000, blit=True)
plt.close()

html = HTML(anim.to_jshtml())
display(html)